# Problema:

O nó relay (retransmissor) de uma rede de comunicação cooperativa possui uma fila com buffer com capacidade para armazenar N pacotes (incluindo o pacote que está em atendimento, ou seja, sendo transmitido pelo relay). Os pacotes recebidos pelo relay são colocados na fila e transmitidos da seguinte forma: se houver um único pacote na fila ele é transmitido imediatamente; se houver dois ou mais pacotes na fila, o relay faz uma operação ou-exclusivo (XOR) entre os dois pacotes, transmite o pacote resultante da operação XOR e retira os dois pacotes utilizados para fazer o XOR da fila. A taxa de chegada de pacotes na fila do relay é λ pacotes/segundo. O enlace de saída do nó relay permite a transmissão de μ pacotes/segundo.

O modelo de lote parcial considera uma fila markoviana com um único servidor e atendimento em lotes. Especificamente, assumimos que os clientes chegam segundo um processo de Poisson ordinário, os tempos de serviço são exponenciais, há um único servidor, os clientes são atendidos na ordem de chegada (FCFS), não há restrição na capacidade de espera, e os clientes são atendidos $K$ de cada vez.
O servidor pode processar lotes parciais de tamanho máximo $K$. Os clientes são atendidos $K$ de cada vez, mas agora, se houver menos de $K$ clientes no sistema, o servidor inicia o atendimento desses clientes.
Além disso, quando há menos de $K$ clientes em atendimento, novas chegadas entram imediatamente no atendimento até o limite $K$ e finalizam junto com os demais, independentemente do tempo de entrada no serviço.
O tempo necessário para o atendimento de qualquer lote é uma variável aleatória com distribuição exponencial de média $1/\mu$, independentemente de o lote estar completo com tamanho $K$ ou não.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

# Implementação da Função

In [ ]:
def simulate_cooperative_relay(
        lamb: float = 10.,
        mu: float = 12.,
        max_buffer_size: int = 5,
        max_departures: int = 50000,
        xor_selection: bool = True,
        seed: int = 42
    ) -> dict:

    """
    Simula um nó relay com buffer limitado em uma rede de comunicação cooperativa.

    O relay opera da seguinte forma:
    - Se houver apenas 1 pacote na fila, ele é transmitido diretamente.
    - Se houver 2 ou mais pacotes e xor_selection for True, realiza-se uma operação XOR
      entre os dois primeiros pacotes da fila e transmite-se o resultado, removendo ambos da fila.
    - Se o buffer estiver cheio no momento de uma nova chegada, o pacote é descartado (bloqueado).

    O processo de chegadas é modelado como um processo de Poisson com taxa `lamb`,
    e o processo de transmissões também segue Poisson com taxa `mu`.

    Args:
        lamb (float): Taxa de chegada de pacotes (λ), em pacotes/segundo.
        mu (float): Taxa de atendimento/transmissão (μ), em pacotes/segundo.
        max_buffer_size (int): Capacidade total do buffer (inclusive o pacote em atendimento).
        max_departures (int): Número máximo de partidas (transmissões) a serem simuladas.
        xor_selection (bool): Se True, permite operação XOR entre dois pacotes na fila.
        seed (int): Semente do gerador de números aleatórios para reprodutibilidade.

    Returns:
        dict: Dicionário com métricas de desempenho do sistema contendo:
            - 'average_system_time' (float): Tempo médio no sistema por pacote.
            - 'average_system_occupancy' (float): Ocupação média do sistema (fila + servidor).
            - 'blck_ratio' (float): Proporção de pacotes bloqueados em relação ao total de chegadas.
    """

    from statistics import mean

    # inicialização de variáveis
    t = 0. # tempo inicial
    len_q = 0 # número de elementos na fila
    server_busy = False # estado inicial do servidor (desocupado)
    j = 0 # número de chegas
    k = 0 # número de partidas
    blocks = 0 # contador de bloqueios

    RNG = np.random.default_rng(seed) # gerador de números aleatórios do numpy

    # listas para armazenar valores
    t_arrival = [] # tempos de chegada já atendidos diretamente
    t_system = [] # tempos no sistema
    t_buffer = [] # fila (buffer) de chegadas que aguardam atendimento
    system_state = [] # guarda tamanho do sistema
    times = [] # armazena os tempos de sistema

    # funções auxiliares
    def gen_arrival(t):
        '''função auxiliar para gerar tempos de chegada'''
        return t + RNG.exponential(1/lamb)

    def gen_departure(t):
        '''função auxiliar para gerar tempos de partida'''
        return t + RNG.exponential(1/mu)

    # gera os primeiros eventos
    next_arrival = gen_arrival(t) # próxima chegada
    next_departure = np.inf # não há partida inicialmente

    # loop principal
    while (k < max_departures): # limitado ao número de partidas

        times.append(t)
        system_state.append(len_q + int(server_busy))

        if next_arrival < next_departure: # evento de chegada

            t = next_arrival # atualiza tempo atual
            j += 1 # incrementa número de chegadas

            if server_busy: # servidor ocupado

                if len_q < max_buffer_size: # se houver espaço no buffer

                    len_q += 1 # incrementa fila
                    t_buffer.append(t) # armazena tempo de chegada no buffer

                else: # buffer cheio

                    blocks += 1 # incrementa bloqueios

            else: # servidor desocupado
                server_busy = True # ocupa o servidor
                t_arrival.append(t) # armazena chegada atendida diretamente
                next_departure = gen_departure(t) # gera próxima partida

            next_arrival = gen_arrival(t) # gera próxima chegada

        else: # evento de partida

            t = next_departure # atualiza tempo atual
            k += 1 # incrementa número de partidas

            if len_q == 0: # não há fila
                t_arrival_eff = t_arrival.pop() # tempo de chegada direta
                server_busy = False # desocupa servidor
                next_departure = np.inf # nenhuma partida agendada

            else: # há fila
                n_withdrawals = 1 if len_q == 1 or not xor_selection else 2 # número de retiradas
                # será 1 se o tamanho da fila for 1 ou se a flag xor_selection for falsa
                len_q -= n_withdrawals # decrementa fila
                t_arrival_eff = [t_buffer.pop(0) for _ in range(n_withdrawals)]
                next_departure = gen_departure(t) # gera nova partida

            # t_departure.append(t) # armazena tempo da partida
            total_time = (t - np.array(t_arrival_eff))
            _ = [t_system.append(item) for item in np.nditer(total_time)]

    average_system_time = np.mean(t_system) # media do tempo de sistema
    average_system_occupancy = np.trapezoid(system_state, times)/times[-1] # número médio de elementos no sistema

    blck_ratio = blocks / j # taxa de bloqueio

    # retorna as variáveis de interesse
    return {
        'average_system_time': average_system_time,
        'average_system_occupancy': average_system_occupancy,
        'blck_ratio': blck_ratio,
    }

# Teste de Sanidade:

Comsiderando o exemplo da Fila M/M/1/n, visto em sala:

$\lambda = 200$ pacotes/s

$\mu = 250$ pacotes/s

$N = 6$

Os Resultados Simulados são compatíveis com a teoria quando `xor_selection=False`?

In [ ]:
lamb = 200
mu = 250
N = 6

In [ ]:
simulate_cooperative_relay(
        lamb=lamb,
        mu=mu,
        max_buffer_size=N,
        max_departures=1000000,
        xor_selection=False,
        seed=42
    )

{'average_system_time': np.float64(0.012561746835178127),
 'average_system_occupancy': np.float64(2.4535951019671125),
 'blck_ratio': 0.05063431741218622}

In [ ]:
def compute_mmk1_queue(lamb, mu, k=2):

    from sympy import symbols, solve, Eq
    from numpy import real, imag

    def absval(x):
        return (real(x)**2 + imag(x)**2)**0.5

    # definir variável simbólica
    r = symbols('r')

    # Resolver a equação (3.8)
    equation = Eq(mu*r**(k+1) - (mu + lamb)*r + lamb, 0)
    solutions = solve(equation, r)
    ro = [root.evalf() for root in solutions if 0<=absval(root)<1][0]

    avg_sys_len = ro / (1 - ro)
    avg_sys_time = ro / lamb

    return {
        'average_system_occupancy': avg_sys_len,
        'average_system_time': avg_sys_time,
    }

In [ ]:
compute_mmk1_queue(lamb, mu)

{'average_system_occupancy': 1.10391256382997,
 'average_system_time': 0.00262347538297980}

In [ ]:
def compoute_mm1j_queue(lamb, mu, N):
    pho = lamb / mu

    p_b = pho**N * (1 - pho) / (1 - pho**(N+1))

    e_q = pho / (1 - pho) - (N + 1)*pho**(N + 1) / (1 - pho**(N+1))

    e_tq = e_q / ((1 - p_b) * lamb)

    return {
        'average_system_time': e_tq,
        'average_system_occupancy': e_q,
        'block_probability': p_b,
    }

In [ ]:
compoute_mm1j_queue(lamb, mu, N+1)

{'average_system_time': 0.01256973486014156,
 'average_system_occupancy': 2.3872477998332764,
 'block_probability': 0.050398506255210127}

Os Resultados São compatíveis!

# Gerar Resultados

In [ ]:
# parâmetros
lamb_values = np.linspace(5, 20, 5)     # Lambda sweep
mu_values = np.linspace(8, 20, 5)       # Mu sweep
buffer_sizes = np.arange(2, 30, 3)       # Buffer size N sweep

default_lamb = 10
default_mu = 12
default_N = 5

def plot_metric(x, y_sim, y_mm1j, y_mmk1, xlabel, ylabel, title, filename):
    plt.figure(figsize=(7, 5))
    plt.plot(x, y_sim, marker='o', label='Simulation', color='black')
    plt.plot(x, y_mm1j, marker='s', linestyle='--', label='$M/M/1/J$', color='blue')
    plt.plot(x, y_mmk1, marker='^', linestyle='-.', label='$M/M^{K}/1$', color='red')

    plt.xlabel(xlabel, fontsize=11)
    plt.ylabel(ylabel, fontsize=11)
    # plt.title(title, fontsize=12)
    plt.grid(True, which='both', linestyle=':', linewidth=0.7)
    plt.legend()
    plt.tight_layout()
    plt.savefig(filename, format='pdf')
    plt.close()

def run_experiment(x_values, param_name):
    results_table = []

    xlabel_map = {
        'lamb': r'$\lambda$ (packets/s)',
        'mu': r'$\mu$ (packets/s)',
        'N': r'$N$ (Buffer Size)'
    }

    metrics = ['average_system_time', 'average_system_occupancy', 'block_probability']
    results_sim = {m: [] for m in metrics}
    results_mm1j = {m: [] for m in metrics}
    results_mmk1 = {m: [] for m in metrics}

    for x in x_values:
        lamb = default_lamb
        mu = default_mu
        N = default_N
        if param_name == 'lamb':
            lamb = x
        elif param_name == 'mu':
            mu = x
        elif param_name == 'N':
            N = int(x)

        # Simulation
        sim_result = simulate_cooperative_relay(lamb, mu, N, max_departures=1000000, xor_selection=False, seed=42)

        # M/M/1/J
        try:
            theo_mm1j = compoute_mm1j_queue(lamb, mu, N+1)
        except ZeroDivisionError:
            theo_mm1j = {'average_system_time': np.nan, 'average_system_occupancy': np.nan, 'block_probability': np.nan}

        # M/M^K/1
        theo_mmk1 = compute_mmk1_queue(lamb, mu, k=2)

        # Append for plotting
        for m in metrics:
            results_sim[m].append(sim_result.get(m if m != 'block_probability' else 'blck_ratio', np.nan))
            results_mm1j[m].append(theo_mm1j.get(m, np.nan))
            results_mmk1[m].append(theo_mmk1.get(m, np.nan) if m != 'block_probability' else 0)

        # Save full row for CSV
        results_table.append({
            'x_value': x,
            'sim_avg_time': sim_result['average_system_time'],
            'mm1j_avg_time': theo_mm1j['average_system_time'],
            'mmk1_avg_time': theo_mmk1['average_system_time'],

            'sim_avg_occupancy': sim_result['average_system_occupancy'],
            'mm1j_avg_occupancy': theo_mm1j['average_system_occupancy'],
            'mmk1_avg_occupancy': theo_mmk1['average_system_occupancy'],

            'sim_blocking': sim_result['blck_ratio'],
            'mm1j_blocking': theo_mm1j['block_probability'],
            'mmk1_blocking': 0,  # No blocking for M/M^K/1
        })

    # Plot metrics
    x = x_values

    plot_metric(x, results_sim['average_system_time'],
                results_mm1j['average_system_time'],
                results_mmk1['average_system_time'],
                xlabel_map[param_name],
                '$E[t_{\mathrm{q}}] (s)$',
                f'Average System Time vs {param_name}',
                f'avg_system_time_vs_{param_name}.pdf')

    plot_metric(x, results_sim['average_system_occupancy'],
                results_mm1j['average_system_occupancy'],
                results_mmk1['average_system_occupancy'],
                xlabel_map[param_name],
                '$E[Q]$ (packets)',
                f'Average System Occupancy vs {param_name}',
                f'avg_occupancy_vs_{param_name}.pdf')

    plot_metric(x, results_sim['block_probability'],
                results_mm1j['block_probability'],
                results_mmk1['block_probability'],
                xlabel_map[param_name],
                '$P_{\mathrm{b}}$',
                f'Blocking Probability vs {param_name}',
                f'blocking_probability_vs_{param_name}.pdf')

    # Save CSV
    df = pd.DataFrame(results_table)
    df.to_csv(f'results_vs_{param_name}.csv', index=False)

    print(f"Saved results_vs_{param_name}.csv and all PDF plots for {param_name} sweep.")

run_experiment(lamb_values, 'lamb')
run_experiment(mu_values, 'mu')
run_experiment(buffer_sizes, 'N')

print("All plots and CSVs generated.")


Saved results_vs_lamb.csv and all PDF plots for lamb sweep.
Saved results_vs_mu.csv and all PDF plots for mu sweep.
Saved results_vs_N.csv and all PDF plots for N sweep.
All plots and CSVs generated.


# Código alternartivo que considera a ocupação do servidor quando os pacotes são retirados da fina $S(t) \in \{0, 1, 2\}$.

In [ ]:
def simulate_cooperative_relay(
        lamb: float = 10.,
        mu: float = 12.,
        max_buffer_size: int = 5,
        max_departures: int = 50000,
        xor_selection: bool = True,
        seed: int = 42
    ) -> dict:


    from statistics import mean

    # inicialização de variáveis
    t = 0. # tempo inicial
    len_q = 0 # número de elementos na fila
    len_s = 0 # estado inicial do servidor (desocupado)
    j = 0 # número de chegas
    k = 0 # número de partidas
    blocks = 0 # contador de bloqueios

    RNG = np.random.default_rng(seed) # gerador de números aleatórios do numpy

    # listas para armazenar valores
    t_arrival = [] # tempos de chegada já atendidos diretamente
    t_system = [] # tempos no sistema
    t_buffer = [] # fila (buffer) de chegadas que aguardam atendimento
    system_state = [] # guarda tamanho do sistema
    times = [] # guarda o tempo de simulação

    # funções auxiliares
    def gen_arrival(t):
        '''função auxiliar para gerar tempos de chegada'''
        return t + RNG.exponential(1/lamb)

    def gen_departure(t):
        '''função auxiliar para gerar tempos de partida'''
        return t + RNG.exponential(1/mu)

    # gera os primeiros eventos
    next_arrival = gen_arrival(t) # próxima chegada
    next_departure = np.inf # não há partida inicialmente

    # loop principal
    while (k < max_departures): # limitado ao número de partidas

        times.append(t)
        system_state.append(len_q + len_s)
        max_queue_size = max_buffer_size - len_s # calcula o número máximo de
        # pacotes que podem ser armazenados na fila

        if next_arrival < next_departure: # evento de chegada

            t = next_arrival # atualiza tempo atual
            j += 1 # incrementa número de chegadas

            if len_s != 0: # servidor ocupado

                if len_q < max_queue_size: # se houver espaço no buffer

                    len_q += 1 # incrementa fila
                    t_buffer.append(t) # armazena tempo de chegada no buffer

                else: # buffer cheio

                    blocks += 1 # incrementa bloqueios

            else: # servidor desocupado
                len_s = 1 # ocupa o servidor
                t_arrival.append(t) # armazena chegada atendida diretamente
                next_departure = gen_departure(t) # gera próxima partida

            next_arrival = gen_arrival(t) # gera próxima chegada

        else: # evento de partida

            t = next_departure # atualiza tempo atual
            k += 1 # incrementa número de partidas

            if len_q == 0: # não há fila
                t_arrival_eff = t_arrival.pop() # tempo de chegada direta
                len_s = 0 # desocupa servidor
                next_departure = np.inf # nenhuma partida agendada

            else: # há fila
                n_withdrawals = 1 if len_q == 1 or not xor_selection else 2 # número de retiradas
                # será 1 se o tamanho da fila for 1 ou se a flag xor_selection for falsa
                len_q -= n_withdrawals # decrementa fila
                t_arrival_eff = [t_buffer.pop(0) for _ in range(n_withdrawals)]
                next_departure = gen_departure(t) # gera nova partida
                len_s = n_withdrawals

            # t_departure.append(t) # armazena tempo da partida
            total_time = (t - np.array(t_arrival_eff))
            _ = [t_system.append(item) for item in np.nditer(total_time)]

    average_system_time = np.mean(t_system) # media do tempo de sistema
    average_system_occupancy = np.trapezoid(system_state, times)/times[-1] # número médio de elementos no sistema

    blck_ratio = blocks / j # taxa de bloqueio

    # retorna as variáveis de interesse
    return {
        'average_system_time': average_system_time,
        'average_system_occupancy': average_system_occupancy,
        'blck_ratio': blck_ratio,
    }